<h2><u> Part 1: Finding the shape of the pre-processed table </u></h2>

In [2]:
import requests
from scrapy.http import TextResponse
from bs4 import BeautifulSoup
import re 
import pandas as pd

<h3> Functions used to parse a website, remove html tags and remove special chars like \n</h3>

In [3]:
#parses web link and returns matches found for given tag
def parser(link, tag):
    res = requests.get(link)
    response = TextResponse(res.url, body=res.text, encoding='utf-8')
    return response.css(tag).getall()
    
#function to remove html tags
def remove_html(movies):
    sol = []
    for movie in movies:
        s = BeautifulSoup(movie, "lxml").text
        if re.search('[a-zA-Z]', s):
            sol.append(s)
    return sol

#function to remove dates and special chars
def remove_non_chars(movies):
    regex = re.compile('[^a-z : - A-Z]')
    for idx, movie in enumerate(movies):
        movies[idx] = regex.sub('', movie)
    return movies


<h3> Parse the website, remove html tags and special chars </h3>

In [4]:
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
tag = 'td'
table = parser(link, tag)
#removed the data outside of the table and remove the html tags scraped 
table = remove_html(table[:-34])
table = remove_non_chars(table)

<h3> Create the pandas dataframe by first grouping every 3 values together in a nested list and then converting it into a pandas dataframe </h3>

In [5]:
#grouping every 3 values a single list inside a list
#easier to make this list and to then append it into a pandas dataframe!
i = 3
lis = []
while i <= len(table):
    lis.append(table[i-3:i])
    i += 3
lis

[['MA', 'Not assigned', 'Not assigned'],
 ['MA', 'Not assigned', 'Not assigned'],
 ['MA', 'North York', 'Parkwoods'],
 ['MA', 'North York', 'Victoria Village'],
 ['MA', 'Downtown Toronto', 'Harbourfront'],
 ['MA', 'North York', 'Lawrence Heights'],
 ['MA', 'North York', 'Lawrence Manor'],
 ['MA', 'Downtown Toronto', 'Queens Park'],
 ['MA', 'Not assigned', 'Not assigned'],
 ['MA', 'Queens Park', 'Not assigned'],
 ['MB', 'Scarborough', 'Rouge'],
 ['MB', 'Scarborough', 'Malvern'],
 ['MB', 'Not assigned', 'Not assigned'],
 ['MB', 'North York', 'Don Mills North'],
 ['MB', 'East York', 'Woodbine Gardens'],
 ['MB', 'East York', 'Parkview Hill'],
 ['MB', 'Downtown Toronto', 'Ryerson'],
 ['MB', 'Downtown Toronto', 'Garden District'],
 ['MB', 'North York', 'Glencairn'],
 ['MB', 'Not assigned', 'Not assigned'],
 ['MB', 'Not assigned', 'Not assigned'],
 ['MB', 'Etobicoke', 'Cloverdale'],
 ['MB', 'Etobicoke', 'Islington'],
 ['MB', 'Etobicoke', 'Martin Grove'],
 ['MB', 'Etobicoke', 'Princess Gardens

In [6]:
#every 3 values needs to be inserted into a pandas dataframe to make the final table
df = pd.DataFrame(lis, columns = ['PostalCode', 'Borough', 'Neighborhood'])
df

,PostalCode,Borough,Neighborhood
0,MA,Not assigned,Not assigned
1,MA,Not assigned,Not assigned
2,MA,North York,Parkwoods
3,MA,North York,Victoria Village
4,MA,Downtown Toronto,Harbourfront
5,MA,North York,Lawrence Heights
6,MA,North York,Lawrence Manor
7,MA,Downtown Toronto,Queens Park
8,MA,Not assigned,Not assigned
9,MA,Queens Park,Not assigned


<h3> Here I drop the not assigned boroughs and reset the index </h3>

In [7]:
#drop cells with not assigned borough
df = df[df.Borough != 'Not assigned']
df.reset_index(drop = True, inplace = True)
df

,PostalCode,Borough,Neighborhood
0,MA,North York,Parkwoods
1,MA,North York,Victoria Village
2,MA,Downtown Toronto,Harbourfront
3,MA,North York,Lawrence Heights
4,MA,North York,Lawrence Manor
5,MA,Downtown Toronto,Queens Park
6,MA,Queens Park,Not assigned
7,MB,Scarborough,Rouge
8,MB,Scarborough,Malvern
9,MB,North York,Don Mills North


<h3> This cell groups by Postal Code and Borough (since some codes have multiple boroughs) and then joins the Neighborhoods together using a comma. I then reset the index to make the PostalCode repeat according to how we want the dataframe to look.</h3>

In [8]:
g = df.reset_index().groupby(['PostalCode', 'Borough'], as_index = False)['Neighborhood'].apply(', '.join)
g = g.to_frame().reset_index()
g.columns = ['PostalCode', 'Borough', 'Neighborhood']
g

,PostalCode,Borough,Neighborhood
0,MA,Downtown Toronto,"Harbourfront, Queens Park"
1,MA,North York,"Parkwoods, Victoria Village, Lawrence Heights,..."
2,MA,Queens Park,Not assigned
3,MB,Downtown Toronto,"Ryerson, Garden District"
4,MB,East York,"Woodbine Gardens, Parkview Hill"
5,MB,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess ..."
6,MB,North York,"Don Mills North, Glencairn"
7,MB,Scarborough,"Rouge, Malvern"
8,MC,Downtown Toronto,St James Town
9,MC,East York,Woodbine Heights


<h3>The code below finds the cell with not assigned neighborhood and assign it to that cells Borough value</h3>

In [16]:
g.loc[g.Neighborhood == 'Not assigned', 'Neighborhood'] = g.loc[g.Neighborhood == 'Not assigned', 'Borough']
print("There are", len(g.loc[g.Neighborhood == 'Not assigned']), "'not assigned neighborhood' cells")

There are 0 'not assigned neighborhood' cells


<h3> Final shape answer </h3>

In [17]:
final_df = g
print("The shape of the fully pre-processed dataframe is: ", final_df.shape)

The shape of the fully pre-processed dataframe is:  (80, 3)
